<a href="https://colab.research.google.com/github/Deep-Parekh/NGramLanguageModel/blob/main/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you will familiarize yourself with:



*   Python
*   Jupyter Notebooks
*   Google Colab

to develop

* A n-gram based language model, with smoothing

to be able to

* Produce naturalish text
* Rate the perplexity of a text given your model

First we will load in some data.

Provided is code that will download a file and rename it corpus.zip

In [1]:

!wget -O corpus.zip https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1&isAllowed=y 

--2021-02-11 08:57:55--  https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1
Resolving digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)... 52.45.246.5
Connecting to digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)|52.45.246.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8536913 (8.1M) [application/octet-stream]
Saving to: ‘corpus.zip’

corpus.zip          100%[===================>]   8.14M  52.1MB/s    in 0.2s    

2021-02-11 08:57:55 (52.1 MB/s) - ‘corpus.zip’ saved [8536913/8536913]



By typing "!" in a notebook, you can use command line prompts

In [2]:
!ls 

corpus.zip  sample_data



Now, we will unzip the corpus


In [3]:
!unzip corpus.zip

Archive:  corpus.zip
  inflating: Shakespeare Corpus by Play/henryVIII/henryVIII_SCRBnoCN_noSD.txt  
  inflating: Shakespeare Corpus by Play/henryVIII/henryVIII_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/macbeth_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/juliusCaesar/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/macbeth_SCRBnoCNnoSD_S.txt  
  inflating: Shakespeare Corpus by Play/henryVIII/_DS_Store  
  inflating: Shakespeare Corpus by Play/juliusCaesar/juliusCaesar_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/edwardIII/_DS_Store  
  inflating: Shakespeare Corpus by Play/henryV/_DS_Store  
  inflating: Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_SCRBnoCNnoSD.txt  
  inflating: Shakespeare Corpus by Play/allsWellThatEndsWell/_DS_Stor

We can even have python and the command line interact.

In [4]:
files = !ls Shake*/*/*_noSCRB.txt 


`files` is now a list containing all of the files we wish to use in our data

In [5]:
print(files)

["'Shakespeare Corpus by Play/allsWellThatEndsWell/allsWellThatEndsWell_noSCRB.txt'", "'Shakespeare Corpus by Play/antonyAndCleopatra/antonyAndCleopatra_noSCRB.txt'", "'Shakespeare Corpus by Play/asYouLikeIt/asYouLikeIt_noSCRB.txt'", "'Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_noSCRB.txt'", "'Shakespeare Corpus by Play/coriolanus/coriolanus_noSCRB.txt'", "'Shakespeare Corpus by Play/cymbeline/cymbeline_noSCRB.txt'", "'Shakespeare Corpus by Play/edwardIII/edwardIII_noSCRB.txt'", "'Shakespeare Corpus by Play/hamlet/hamlet_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVparti/henryIVparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVpartii/henryIVpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryV/henryV_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIII/henryVIII_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIparti/henryVIparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartii/henryVIpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartiii/h

Note, there are extra quotes on the files, so we will use list comprehension to remove those

In [6]:
files = [file[1:-1] for file in files]

#STEP 1


* Go through all of the files
  * Read them in
  * Iterate through them line by line
  * Split each line by white space
  * Make sure to add in a special newline token `'\n'` for every line
  * Every file should be in its own list
  * Add special start `'<s>'` and end `'</s>'` tokens to the list for each file
  * The end results should be an list of lists of strings called `texts`



  e.g.

  If we read in one file that looked like:
  
    HAMLET: Alas poor Yorick.

    GRAVEDIGGER: Put that skull down.



  The end results should look like:

  `texts = [['<s>', 'HAMLET:', 'Alas', 'poor', 'Yorick.','\n','GRAVEDIGGER:', Put','that', 'skull', 'down.', '</s>']]`
 

In [7]:
texts = []
for txtfile in files:
  with open(txtfile,'r') as txt:
    per_file = ['<s>']
    for line in txt:
      for token in  line.split(' '):
        if len(token) > 0:
          per_file.append(token)
    per_file.append('</s>')
  texts.append(per_file)
  

In [8]:
n = 2
bigrams = {}
for text in texts:
  for index in range(len(text)-(n-1)):
    bigram = tuple(text[index:index+n])
    bigrams[bigram] = bigrams.get(bigram,0) + 1

In [9]:
print(list(bigrams.items())[:50])

[(('<s>', '1603\n'), 1), (('1603\n', '\n'), 1), (('\n', 'ALLS'), 1), (('ALLS', 'WELL'), 1), (('WELL', 'THAT'), 1), (('THAT', 'ENDS'), 1), (('ENDS', 'WELL\n'), 1), (('WELL\n', '\n'), 1), (('\n', 'by'), 37), (('by', 'William'), 37), (('William', 'Shakespeare\n'), 37), (('Shakespeare\n', '\n'), 37), (('\n', '\n'), 2756), (('\n', 'Dramatis'), 20), (('Dramatis', 'Personae\n'), 19), (('Personae\n', '\n'), 19), (('\n', 'KING'), 254), (('KING', 'OF'), 4), (('OF', 'FRANCE\n'), 4), (('FRANCE\n', 'THE'), 1), (('THE', 'DUKE'), 3), (('DUKE', 'OF'), 103), (('OF', 'FLORENCE\n'), 1), (('FLORENCE\n', 'BERTRAM,'), 1), (('BERTRAM,', 'Count'), 1), (('Count', 'of'), 1), (('of', 'Rousillon\n'), 4), (('Rousillon\n', 'LAFEU,'), 1), (('LAFEU,', 'an'), 1), (('an', 'old'), 67), (('old', 'lord\n'), 1), (('lord\n', 'PAROLLES,'), 1), (('PAROLLES,', 'a'), 1), (('a', 'follower'), 1), (('follower', 'of'), 6), (('of', 'Bertram\n'), 1), (('Bertram\n', 'TWO'), 1), (('TWO', 'FRENCH'), 1), (('FRENCH', 'LORDS,'), 4), (('LOR

#STEP 2

With our files read in, it's time to do some counting!

* Implement the function `get_counts(texts,n)` which takes in a list of list of strings -- `texts` -- and the arity of the n-gram -- `n` 
* `get_counts` should return a dictionary where the keys are the n-grams (as tuples of strings) and the values are the integer counts of all of the found n-grams


Here's a quick primer on tuples in Python

    triple = (1,2,3)
    double = (1,2)
    single = (1,) #note the trailing comma to distinguish from parenethesis 
    null_tuple = tuple() 

    list_triple = [1,2,3]
    tuple_triple = tuple(list_triple)
    


In [24]:
def get_counts(texts,n):
  ngrams = {}
  for text in texts:
    for index in range(len(text)-(n-1)):
      ngram = tuple(text[index:index+n])
      ngrams[ngram] = ngrams.get(ngram,0) + 1
  print(list(ngrams)[:50])
  return ngrams


#STEP 3

With our counts, it's time to make our first language model.

* Implement the class NGramLM 
  * Initialize the LM
    * First use `get_counts` to get the raw counts of the n-grams up to order `n`
    * Convert these raw counts into probabilities
      * ### Question 1:  What is the relationship between raw counts of n-grams of order n and n-1 and the probabilities of the next word for order n? 
  * Implement the `generate_text` function
    * You are given an optional prompt (a list of strings)
    * You should return a list of strings based on randomly choosing according to the probabilities of the language model (using the `random` library)
    * Your function should fail gracefully if an n-gram can't be found by backing off
  * Implement `score_text` which when given a text (a list of strings) returns the perplexity of that text
    * Make sure you don't have any divide by 0 (or log(0)) issues -- return float('inf') if there is an n-gram not found in the model
    * Use backoff to try to somewhat gracefully handle issues

In [25]:
import random
random.seed(0)

class NGramLM():
  def __init__(self,texts,n):
    self.n = n
    self.context = list()    # Keeps track of what words have been seen
    unigram = get_counts(texts,1)
    self.unigram_probabilities = get_probabilities(unigram)
    self.probabilities = {}
    for i in range(2,n+1):
      ngram = get_counts(texts,i)
      self.probabilities = {**get_probabilities(ngram),**self.probabilities}

    print(list(self.unigram_probabilities)[:50])

  def generate_text(self, length,prompt=[]):
    text = ""
    if not prompt:
      self.context.append("<s>")
      text += "<s>"
    for i in range(length):
      # Backing off enabled
      n = (self.n-1) if len(self.context) >= self.n else len(self.context)+1
      for j in range(n-1,0,-1):
        if j == 1:
          rand = random.random()
          for word, prob in self.unigram_probabilities:
            if prob < rand:
              rand -= prob
            else:
              text += word
              self.context.append(word)
          continue
        possible_words = self.probabilities[tuple(self.context[-j:])]
        if not possible_words:
          continue
        total = sum(k for _, k in possible_words.items())
        rand = random.random()
        for word, prob in possible_words:
          if prob < rand:
            rand -= prob
          else:
            text += word
            self.context.append(word)
      print(text)
    return text

  def score_text(self,text):
    raise NotImplementedError("STEP 3: Given a text, return the perplexity of that text ")

def get_probabilities(counts):
  probability = dict()
  n = len(list(counts.keys())[0])
  if n is 1:
    print("Counting unigram probs")
    total = sum(counts.values())
    for word, count in counts.items():
      probability[word] = count/total
    return probability

  for ngram, count in counts.items():
    probability[ngram[0:n-1]] = probability.get(ngram[0:n-1], [])
    probability[ngram[0:n-1]].append([ngram[-1], count])

  for ngram, following_words in probability.items():
    total = sum(i for _, i in following_words)
    for word in following_words:
      word[1] = float(word[1])/total

  return probability

lm = NGramLM(texts,4)
#lm.generate_text(100)

[('<s>',), ('1603\n',), ('\n',), ('ALLS',), ('WELL',), ('THAT',), ('ENDS',), ('WELL\n',), ('by',), ('William',), ('Shakespeare\n',), ('Dramatis',), ('Personae\n',), ('KING',), ('OF',), ('FRANCE\n',), ('THE',), ('DUKE',), ('FLORENCE\n',), ('BERTRAM,',), ('Count',), ('of',), ('Rousillon\n',), ('LAFEU,',), ('an',), ('old',), ('lord\n',), ('PAROLLES,',), ('a',), ('follower',), ('Bertram\n',), ('TWO',), ('FRENCH',), ('LORDS,',), ('serving',), ('with',), ('STEWARD,',), ('Servant',), ('to',), ('the',), ('Countess',), ('LAVACHE,',), ('clown',), ('and',), ('A',), ('PAGE,',), ('COUNTESS',), ('ROUSILLON,',), ('mother',), ('HELENA,',)]
Counting unigram probs
[('<s>', '1603\n'), ('1603\n', '\n'), ('\n', 'ALLS'), ('ALLS', 'WELL'), ('WELL', 'THAT'), ('THAT', 'ENDS'), ('ENDS', 'WELL\n'), ('WELL\n', '\n'), ('\n', 'by'), ('by', 'William'), ('William', 'Shakespeare\n'), ('Shakespeare\n', '\n'), ('\n', '\n'), ('\n', 'Dramatis'), ('Dramatis', 'Personae\n'), ('Personae\n', '\n'), ('\n', 'KING'), ('KING', 'O

In [ ]:
trigramLM = NGramLM(texts,3)

print(' '.join(trigramLM.generate_text(100)))
print(' '.join(trigramLM.generate_text(100,['HAMLET.'])))
print(' '.join(trigramLM.generate_text(100,['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss'])))
print(' '.join(trigramLM.generate_text(100,['Hamlet','just','use','Google'])))


print(trigramLM.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
print(trigramLM.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss']))
print(trigramLM.score_text(['what','do','you','say','to','that','Hamlet']))

#STEP 4

Build a better language model

* Implement the function `truncate_texts` which takes in texts and a threshold
  * Replace all words with counts less than the threshold with `'<UNK>'`
* Implement the function `get_vocabulary` which takes in texts and returns a set of all of the words found in the texts
* Implement the LaplaceLM language model
  * Use add-alpha Laplace smoothing to calculate the probabilities
  * Implement the `generate_text` and `score_text` functions as above EXCEPT
    * Handle unknown vocabulary gracefully by checking the vocabulary and replacing with `'<UNK>'`

In [ ]:
def truncate_texts(texts,threshold):
  modified_texts = []
  unigrams = get_counts(texts, 1)
  for unigram in unigrams:
    if unigram[1] < threshold:
      unigrams['<UNK>'] = unigrams.get('<UNK>', 0) + unigram[1]
  return modified_texts

def get_vocabulary(texts):
  vocab = set()
  for text in texts:
    for word in text:
      vocab.add(word)
  return vocab

class LaplaceLM(NGramLM):
  def __init__(self,texts,n,vocabulary, alpha=1):
    raise NotImplementedError("STEP 4: Initialize the LM using add-alpha smoothing")
 
  def generate_text(self, length,prompt=[]):
    raise NotImplementedError("STEP 4: Given a prompt (a list of strings) generate length number of strings -- return a list containing all of the text (prompt + generated) ")

  def score_text(self,text):
    raise NotImplementedError("STEP 4: Given a text, return the perplexity of that text ")


truncated = truncate_texts(texts,2)
vocab = get_vocabulary(truncated)
for alpha in [0,1e-5,0.1,1,10,1000]:
  print(f'LM with alpha of {alpha}')
  trigramLM = LaplaceLM(truncated,3,vocab,alpha)

  print(' '.join(trigramLM.generate_text(100)))
  print(' '.join(trigramLM.generate_text(100,['HAMLET.'])))
  print(' '.join(trigramLM.generate_text(100,['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss'])))
  print(' '.join(trigramLM.generate_text(100,['Hamlet','just','use','Google'])))


  print(trigramLM.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
  print(trigramLM.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers', 'kiss']))
  print(trigramLM.score_text(['what','do','you','say','to','that','Hamlet']))

### Question 2: What do you notice with the changing of the alpha?  How do perplexities change?  How does the quality of the generated text change? How do they differ from the simple LM?



Bonus time:

#### 2 points:  Find 2 text datasets (they can be of your own creation, pre-existing datasets, text you scrape, etc.) and combine them together to make an amusing bot.  

e.g. look at https://kingjamesprogramming.tumblr.com/ which combines the bible with Structure and Interpretation of Computer Programs

Showcase your best efforts below

#### 8 points: Make a LM like above except use Kneser-Ney smoothing.  Show it's results and discuss how they differ from the above models
